In [12]:
%matplotlib inline

# import keras
import keras
from keras import backend as K 

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu
from keras_retinanet.losses import smooth_l1, focal
from keras_retinanet.preprocessing import csv_generator
from keras_retinanet.utils.eval import evaluate

# import hyperopts
from hyperopt import fmin, tpe, hp, anneal, Trials

# other imports
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

setup_gpu(1)

2 Physical GPUs, 1 Logical GPUs


In [3]:
default_params = {
    N: 3,
    M: 10
}
val_annotations = Path('/data/students_home/fschipani/thesis/MSc-Thesis-PJ/Dataset/KAIST_MPD/imageSets/csv_files_NO_PEOPLE/lwir/val-all-01-no-people.csv')
validation_generator = csv_generator.CSVGenerator(
    val_annotations,
    classes,
    shuffle_groups=False,
    auto_augment=None,
    rand_augment=None,
    config=None
            )
def create_train_generator(N, M):
    train_generator = csv_generator.CSVGenerator(
        annotations,
        classes,
        transform_generator=None,
        visual_effect_generator=None,
        image_min_side=800,
        image_max_side=1333,
        auto_augment=None,
        rand_augment=(N,M),
        config=None
        )
    return train_generator
train_generator = create_train_generator(default_params[N], default_params[M])

In [5]:
def create_model():
    model_path = Path('/data/students_home/fschipani/thesis/keras-retinanet/snapshots/resnet50_coco_best_v2.1.0.h5')
    model = models.backbone('resnet50').retinanet(num_classes=train_generator.num_classes())
    model.load_weights(model_path, by_name=True, skip_mismatch=True)
    model.compile(
        loss={
            'regression'    : smooth_l1(),
            'classification': focal()
        },
        optimizer=keras.optimizers.adam(lr=1e-5, clipnorm=0.001)
    )
    return model

In [7]:
def test_model(model):
    result = evaluate(
        validation_generator,
        model,
        score_threshold=0.3,
        iou_threshold=0.5,
        max_detections=100,
        save_path=None
    )
    return result[result.keys()[-2]]

    

3

In [13]:
search_space = {
    'N': hp.choice('N', np.arange(1, 3, 1)), #[1-3]
    'M': hp.quniform('M', low=5, high=30, q=1), #[5-30]
}
def fcn_to_optimize(params):
    model = create_model()
    train_generator = create_train_generator(params['N'], params['M'])
    mAP = test_model(model)
    K.clear_session()
    return {
        'mAP': -mAP,
        'status': STATUS_OK
    }


In [ ]:
trials = Trials()
best = fmin(fcn_to_optimize, search_space, algo=tpe.suggest, max_evals=1)


In [ ]:
plt.figure()
xs = [t['tid'] for t in trials.trials]
ys = [-t['result']['mAP'] for t in trials.trials]
plt.xlim(xs[0]-1, xs[-1]+1)
plt.scatter(xs, ys, s=20, linewidth=0.01, alpha=0.75)
plt.xlabel('Iteration', fontsize=16)
plt.ylabel('Accuracy', fontsize=16)
plt.show()